In [4]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Innovation CBC Slide Duplicate\Innovation CBC Replacement Function.ipynb"

### Parameters

In [ ]:
filePath = r"C:\Users\Salma Hany\Documents\Slide-Automate\Innovation CBC Slide Duplicate\Carrefour inno data.xlsx"
data_source = "DATA SOURCE: Consumer Test | April 2025"
market='Carrefour'
sheet_name1='Performance'
sheet_name2='Ranking'
wdtext="WD assumption = Sector Avg"

### Reading Data in Excel

In [6]:
performance = pd.read_excel(filePath,'Performance').rename(columns={'Unnamed: 2': 'Product', 'Unnamed: 3': 'ProductSize'})
rank = pd.read_excel(filePath,'Ranking')
sourcing = pd.read_excel(filePath,"Sourcing")


### Data Cleaning

In [65]:
numberOfSlides=performance.Grouping.value_counts().reset_index()
numberOfSlides['Count']=numberOfSlides['count'].apply(lambda x : 1 if x<=5 else ((x//5) +( 0 if x%5 ==0 else 1)) )

In [66]:
columnName = performance.groupby('Grouping')['Sheets names'].unique().reset_index()

# Find the first column name matching the pattern 'New ... share'
share_cols = [col for col in rank.columns if re.match(r'^New .* Share$', col, re.IGNORECASE)]
if share_cols:
    col = share_cols[0]  # use the first match
    # print(col)
    dfDic = {
        columnName['Grouping'].iloc[i]: rank[[col] + list(columnName['Sheets names'].iloc[i])]
        for i in range(columnName.shape[0])}
   
# Initializing an empty dictionary to store split DataFrames
dfDicSplit = {}

# Column name to be prioritized in the final DataFrames
# col = 'New volume share'

    
# Iterating over each key-value pair in dfDic
for key, value in dfDic.items():
    # Checking if the number of columns (excluding 'New volume share') is greater than 5
    if value.shape[1] - 1 > 5:
        numOfColumns = 0
        # Splitting the DataFrame into chunks of 5 columns if there are more than 5 columns
        while numOfColumns < value.shape[1] - 1:
            print('split number = ', numOfColumns)
            # Creating a new DataFrame with the next set of 5 columns
            dfDicSplit[key + '_Num ' + str(numOfColumns)] = value.iloc[:, numOfColumns + 1:numOfColumns + 6]
            # Adding the 'New volume share' column to the new DataFrame
            dfDicSplit[key + '_Num ' + str(numOfColumns)][col] = value[col]
            # Reordering columns to place 'New volume share' at the front
            columns_order = [col] + [cl for cl in dfDicSplit[key + '_Num ' + str(numOfColumns)].columns if cl != col]
            dfDicSplit[key + '_Num ' + str(numOfColumns)] = dfDicSplit[key + '_Num ' + str(numOfColumns)].reindex(columns=columns_order)
            numOfColumns += 5
    else:
        # If the number of columns (excluding 'New volume share') is 5 or fewer, add the DataFrame as is to dfDicSplit
        dfDicSplit[key] = value


In [67]:
# Creating a dictionary to store the count of columns (excluding 'New volume share' and the 'Grouping' column)
# for each DataFrame in dfDicSplit
duplicationList={key:value.shape[1]-1 for key,value in dfDicSplit.items()}
duplicationList

{'Advanced Protection': 1,
 'Baby': 1,
 'Beautycare': 4,
 'Kids': 1,
 'Ozono': 3,
 'Silk Hydration': 3,
 'Sport': 4}

In [109]:
# Grouping the 'performance' DataFrame by the 'Grouping' column and aggregating 'Sheets names' and 'Product' into lists
grouped = performance.groupby('Grouping')[['Sheets names', 'Product']].agg(list)

# Creating a dictionary where each key is a 'Grouping' value and each value is a dictionary 
# with 'Sheets names' as keys and 'Product' as values
result = grouped.apply(lambda x: dict(zip(x['Sheets names'], x['Product'])), axis=1).to_dict()

# Initializing an empty list to store the number of products minus one for each grouping
sum_list = []

# Iterating over each inner dictionary in the result dictionary
for inner_dict in result.values():
    # Calculating the number of products for the current grouping
    inner_sum = len(inner_dict.values())
    # Appending the number of products minus one to sum_list
    sum_list.append(inner_sum - 1)


### Duplication Stage

In [110]:
list(duplicationList.values())

[1, 1, 4, 1, 3, 3, 4]

In [111]:
startIndex=8
index = [0,list(duplicationList.values()),7,[i+startIndex for i in sum_list]]
duplication = [1,1,performance.Grouping.nunique(),1]
section_names = ["Innovation Summary","Innovation ranking","Innovation potential","Innovation sourcing"]

path = os.getcwd() + '//Inno CBC base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'

In [134]:
print(len(index))
print(len(duplication))
print(len(section_names))
print(index)
print(duplication)
print(section_names)
print(sum(duplication))


4
4
4
[0, [1, 1, 4, 1, 3, 3, 4], 7, [8, 8, 11, 8, 10, 10, 11]]
[1, 1, 7, 1]
['Innovation Summary', 'Innovation ranking', 'Innovation potential', 'Innovation sourcing']
10


In [157]:
## Shouldn't be hased in the first run
slideDuplication(index,duplication,section_names,path,new_pre)

In [158]:
prs = Presentation(new_pre)

### Slide 1

In [159]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [161]:
p=0
Innovation_Summary(prs, performance,numOfDuplicates=duplication[p], position=calculate_position(p))
p+=1

In [162]:
numOfDuplicatesFirstSlide = len(list(duplicationList.values())) * duplication[p]

innovationRanking(prs,dfDicSplit,performance,numOfDuplicates=numOfDuplicatesFirstSlide,position=calculate_position(p))
p+=1

Advanced Protection
Baby
Beautycare
Kids
Ozono
Silk Hydration
Sport


### Slide 2

In [164]:
# positionOfSecondSlide = numOfDuplicatesFirstSlide 
innovationPotential(prs,performance, wdtext,numOfDuplicates = duplication[p],position=calculate_position(p))
p+=1

In [165]:
def innovationSourcing(prs, performance, sourcing, result, sumlist, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with sourcing analysis based on provided data.

    Parameters:
    prs (Presentation): PowerPoint presentation object.
    performance (DataFrame): DataFrame containing performance data.
    sourcing (DataFrame): DataFrame containing sourcing data.
    result (dict): Dictionary mapping groups to products.
    sumlist (list): List of sums indicating the number of products per group.
    numOfDuplicates (int): Number of duplicates to process.
    position (int, optional): Starting position of the slide to update. Default is 0.
    """
    for group, slide_num in zip(result, range(len(sumlist))):
        # Access the slide based on the current position and slide number
        slide = prs.slides[slide_num + position]
        shapes = slide.shapes
        tables, charts = createTableAndChart(shapes)
        
        # Define the market and update the title and data source text
        titleNumber = get_shape_number(shapes, "Innovation | Sourcing Analysis | Rescaled Delta Share by Product | Panda | Cookies'n Creme innovation")
        dataSourceNumber = get_shape_number(shapes, "DATA SOURCE: Consumer Test | Dec 2023")
        shapes[dataSourceNumber-1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Panda', market).replace("Cookies'n Creme innovation", group)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Prepare the DataFrame for chart data
        # print(sourcing.columns)
        new = sourcing[['Rescaled share diff'] + list(result[group].keys())]
        new['sum'] = new[list(result[group].keys())].apply(lambda x: sum(x), axis=1)
        new_sorted = new.sort_values(by='sum', ascending=True).iloc[:11]

        # Update the charts with new data
        for i, (key, value) in zip(range(len(list(result[group].keys()))), result[group].items()):

            chart_data = CategoryChartData()
            chart_data.categories = new_sorted['Rescaled share diff']
            chart_data.add_series(value, new_sorted.iloc[:, i+1] * -1)
            charts[i].chart.replace_data(chart_data)

        # Update the table with new data
        for row_number, row in enumerate(tables[0].table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num > 0:
                    # Update the header row with product names
                    value = list(result[group].values())[column_num - 1]
                    cell.text = str(value)
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                    cell.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(87, 85, 85)
                if row_number > 0 and column_num == 0:
                    # Update the first column with Rescaled share diff values
                    value = new_sorted.iloc[row_number - 1, column_num]
                    cell.text = str(value)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                    cell.text_frame.paragraphs[0].runs[0].font.size = Pt(7)
                    cell.text_frame.paragraphs[0].runs[0].font.bold = False
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'


### Side 3

In [166]:
# numOfDuplicatesThirdSlide = numOfDuplicatesFirstSlide + duplication[1]
# numOfDuplicatesThirdSlide = len(list(sum_list)) * duplication[2]
# positionOfThirdSlide = positionOfSecondSlide + duplication[1]
innovationSourcing(prs,performance, sourcing,result, sum_list, numOfDuplicates = duplication[p], position=calculate_position(p))
p+=1

In [167]:
from datetime import date
today = str(date.today())
outputPath=os.getcwd() + "\\Innovation CBC "+market+" output "+today+".pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)